In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import os

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
path ='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data/processed'

sales_data = pd.read_csv(f'{path}/df_pred.csv')

In [4]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47107050 entries, 0 to 47107049
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   item_id      object 
 1   store_id     object 
 2   date         object 
 3   total_sales  float64
 4   day_of_week  int64  
 5   month        int64  
 6   year         int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.5+ GB


In [5]:
sales_data.head()

,item_id,store_id,date,total_sales,day_of_week,month,year
0,HOBBIES_1_001,CA_1,2011-01-29,0.0,5,1,2011
1,HOBBIES_1_002,CA_1,2011-01-29,0.0,5,1,2011
2,HOBBIES_1_003,CA_1,2011-01-29,0.0,5,1,2011
3,HOBBIES_1_004,CA_1,2011-01-29,0.0,5,1,2011
4,HOBBIES_1_005,CA_1,2011-01-29,0.0,5,1,2011


In [6]:
columns_to_drop = ['day_of_week', 'month', 'year']

sales_data = sales_data.drop(columns=columns_to_drop)

In [7]:
ca_1 = sales_data[sales_data['store_id'] == 'CA_1']
ca_1.head()

,item_id,store_id,date,total_sales
0,HOBBIES_1_001,CA_1,2011-01-29,0.0
1,HOBBIES_1_002,CA_1,2011-01-29,0.0
2,HOBBIES_1_003,CA_1,2011-01-29,0.0
3,HOBBIES_1_004,CA_1,2011-01-29,0.0
4,HOBBIES_1_005,CA_1,2011-01-29,0.0


In [8]:
from sklearn.model_selection import train_test_split

# Splitting the features and target variable
X = ca_1.drop(columns=['total_sales'])  # Features
y = ca_1['total_sales']  # Target

# Train-validation-test split (80% train, 20% test, then 20% of train for validation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Identifying categorical and numerical features
categorical_features = X.select_dtypes(include=['category', 'object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing: OneHotEncoding for categorical features and StandardScaler for numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Creating a RandomForestRegressor pipeline
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
rf_pipeline.fit(X_train, y_train)

In [ ]:
from my_krml_25076922.models.performance import print_regressor_scores

print_regressor_scores(y_preds=y_base, y_actuals=y_train, set_name='Training')